# Análisis de Redes Sociales en Montañismo Chileno: Procesamiento Avanzado con Modelos de Lenguaje y LLMs

**Estudiante:** Francisco Javier Peñailillo San Martín  
**Curso:** Análisis de Redes Sociales  
**Programa:** Magíster en Tecnologías de la Información  
**Universidad:** Universidad Técnica Federico Santa María  
**Fecha:** Julio 2025

---

## 1. Resumen Ejecutivo

Este notebook implementa una solución integral para el análisis de redes sociales en montañismo chileno utilizando técnicas avanzadas de procesamiento de lenguaje natural (NLP) y modelos de lenguaje grandes (LLMs). A partir de datos estructurados de rutas de montaña, aplicamos análisis de sentimientos, extracción de entidades, y construcción de redes sociales basadas en contenido textual para identificar patrones de colaboración, influencia y evolución en la comunidad montañista chilena.

## 2. Herramientas y Técnicas de Procesamiento de Lenguaje Natural

### 2.1 Análisis de los Datos Disponibles

Nuestros datos provienen de la tabla `andes_handbook_routes` que contiene:
- **Nombres de rutas**: Con información geográfica y altitud
- **Descripciones detalladas**: Contenido rico en información social y técnica
- **Metadatos estructurados**: Ubicación, dificultad, características

### 2.2 Stack Tecnológico para NLP y LLM

#### Modelos de Lenguaje Natural
- **Transformers (Hugging Face)**: Modelos pre-entrenados especializados
- **BERT Multilingüe**: Para análisis de entidades nombradas
- **RoBERTa en Español**: Para análisis de sentimientos localizado
- **Databricks LLM (Llama 4)**: Para análisis semántico avanzado

#### Procesamiento y Análisis
- **PySpark**: Procesamiento distribuido de texto a gran escala
- **NetworkX**: Construcción de redes sociales basadas en texto
- **Pandas**: Manipulación de resultados estructurados
- **Delta Lake**: Almacenamiento versionado de análisis

#### Especialización en Montañismo
- **Diccionarios especializados**: Terminología técnica de montañismo
- **Patrones de extracción**: Regex optimizados para nombres y colaboraciones
- **Análisis contextual**: Comprensión de jerga y expresiones específicas

## 3. Instalación de Dependencias y Configuración

In [0]:
# Instalación de librerías especializadas para NLP
%pip install transformers torch
%pip install networkx matplotlib seaborn plotly
%pip install openai
%pip install scipy scikit-learn
%pip install wordcloud

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## 4. Importación de Librerías y Configuración del Entorno

In [0]:
import pandas as pd
import numpy as np
import re
import json
import time
from datetime import datetime
from typing import List, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

# NLP y modelos de lenguaje
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification, 
    AutoModelForSequenceClassification, pipeline
)
import torch
from scipy.special import softmax

# Análisis de redes y visualización
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from wordcloud import WordCloud

# Configuración de Spark y Delta
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf, when, regexp_extract, split, explode

# LLM Integration
from openai import OpenAI
import os

print("🧠 ANDESHANDBOOK NLP & LLM SOCIAL NETWORK ANALYZER")
print("Sistema Avanzado de Análisis de Redes Sociales basado en Procesamiento de Lenguaje Natural")
print("="*90)

🧠 ANDESHANDBOOK NLP & LLM SOCIAL NETWORK ANALYZER
Sistema Avanzado de Análisis de Redes Sociales basado en Procesamiento de Lenguaje Natural


## 5. Configuración de Modelos de Lenguaje

In [0]:
class MountaineeringNLPEngine:
    """
    Motor de NLP especializado para análisis de contenido de montañismo
    """
    
    def __init__(self):
        self.models = {}
        self.setup_models()
        self.mountaineering_vocab = self.load_specialized_vocabulary()
        
    def setup_models(self):
        """Inicializa todos los modelos de NLP"""
        print("🔧 Inicializando modelos de lenguaje...")
        
        try:
            # 1. Modelo de análisis de sentimientos en español
            print("📊 Cargando modelo de sentimientos...")
            self.models['sentiment_tokenizer'] = AutoTokenizer.from_pretrained("Manauu17/enhanced_roberta_sentiments_es")
            self.models['sentiment_model'] = AutoModelForSequenceClassification.from_pretrained("Manauu17/enhanced_roberta_sentiments_es")
            
            # 2. Modelo de reconocimiento de entidades nombradas
            print("🏷️ Cargando modelo NER...")
            self.models['ner_tokenizer'] = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
            self.models['ner_model'] = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
            self.models['ner_pipeline'] = pipeline("ner", 
                                                   model=self.models['ner_model'], 
                                                   tokenizer=self.models['ner_tokenizer'],
                                                   aggregation_strategy="simple")
            
            # 3. Configuración de LLM (Databricks)
            print("🚀 Configurando LLM...")
            try:
                DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
                self.models['llm_client'] = OpenAI(
                    api_key=DATABRICKS_TOKEN,
                    base_url="https://dbc-6f162706-efdf.cloud.databricks.com/serving-endpoints"
                )
                print("✅ LLM configurado exitosamente")
            except Exception as e:
                print(f"⚠️ LLM no disponible: {e}")
                self.models['llm_client'] = None
            
            print("✅ Todos los modelos inicializados correctamente")
            
        except Exception as e:
            print(f"❌ Error inicializando modelos: {e}")
            
    def load_specialized_vocabulary(self) -> Dict[str, List[str]]:
        """Carga vocabulario especializado en montañismo"""
        return {
            'mountaineers_indicators': [
                'ascendido por', 'realizado por', 'primer ascenso', 'primera ascensión',
                'escalado por', 'conquistado por', 'logrado por', 'completado por'
            ],
            'collaboration_keywords': [
                'junto a', 'acompañado de', 'en compañía de', 'con', 'y', 'equipo',
                'cordada', 'expedición', 'grupo', 'dupla', 'trío'
            ],
            'risk_factors': [
                'peligroso', 'riesgo', 'cuidado', 'atención', 'avalancha', 'alud',
                'caída de rocas', 'mal tiempo', 'exposición', 'técnico', 'glaciar'
            ],
            'difficulty_terms': [
                'fácil', 'moderado', 'difícil', 'muy difícil', 'extremo',
                'grado', 'nivel', 'dificultad', 'técnico', 'físico'
            ],
            'emotional_indicators': [
                'espectacular', 'hermoso', 'impresionante', 'desafiante', 'gratificante',
                'frustrante', 'agotador', 'emocionante', 'inolvidable', 'recomendado'
            ]
        }

# Inicializar motor NLP
nlp_engine = MountaineeringNLPEngine()

🔧 Inicializando modelos de lenguaje...
📊 Cargando modelo de sentimientos...


tokenizer_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## 6. Extracción y Preparación de Datos

In [0]:
def load_mountaineering_data():
    """
    Carga datos de rutas desde la tabla principal
    """
    print("📊 Cargando datos de rutas de montaña...")
    
    # Query principal para extraer datos estructurados
    query = """
    SELECT *
    FROM andes_handbook_routes 
    WHERE 
        name != 'Andeshandbook - Guia de Montanismo, Trekking y Senderismo'
        AND description IS NOT NULL 
        AND description != ''
        AND length(description) > 50
    ORDER BY route_id
    """
    
    df = spark.sql(query)
    total_routes = df.count()
    
    print(f"✅ Cargadas {total_routes} rutas con descripciones válidas")
    
    # Convertir a Pandas para procesamiento NLP
    pandas_df = df.toPandas()
    
    # Limpieza básica de texto
    pandas_df['description_clean'] = pandas_df['description'].apply(clean_text_for_nlp)
    pandas_df['name_clean'] = pandas_df['name'].apply(clean_text_for_nlp)
    
    print(f"📝 Texto preparado para análisis NLP")
    
    return pandas_df

def clean_text_for_nlp(text: str) -> str:
    """
    Limpieza específica para análisis NLP
    """
    if not text or pd.isna(text):
        return ""
    
    text = str(text)
    
    # Remover HTML tags si existen
    text = re.sub(r'<[^>]+>', '', text)
    
    # Normalizar espacios
    text = re.sub(r'\s+', ' ', text)
    
    # Remover caracteres especiales excesivos
    text = re.sub(r'[^\w\s\.\,\;\:\!\?\(\)\-\']', ' ', text)
    
    # Limitar longitud para modelos
    text = text[:2000]  # Máximo 2000 caracteres
    
    return text.strip()

# Cargar datos
routes_df = load_mountaineering_data()
print(f"📊 Dataset preparado: {len(routes_df)} rutas listas para análisis")

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.data

## 7. Análisis de Sentimientos Especializado

In [0]:
class SentimentAnalyzer:
    """
    Analizador de sentimientos especializado para contenido de montañismo
    """
    
    def __init__(self, nlp_engine):
        self.nlp_engine = nlp_engine
        self.sentiment_results = []
        
    def analyze_route_sentiment(self, text: str, route_id: str = None) -> Dict[str, Any]:
        """
        Análisis de sentimientos para descripciones de rutas
        """
        if not text or len(text.strip()) < 10:
            return self._empty_sentiment_result()
        
        try:
            # Truncar texto para el modelo
            text_input = text[:512]
            
            tokenizer = self.nlp_engine.models['sentiment_tokenizer']
            model = self.nlp_engine.models['sentiment_model']
            
            # Tokenización y predicción
            encoded_input = tokenizer(
                text_input, 
                return_tensors='pt', 
                padding=True, 
                truncation=True, 
                max_length=512
            )
            
            with torch.no_grad():
                output = model(**encoded_input)
                scores = output.logits.detach().numpy()
            
            # Convertir a probabilidades
            probabilities = softmax(scores, axis=1)[0]
            
            # Mapear labels
            labels = list(model.config.id2label.values())
            scores_dict = {label: float(prob) for label, prob in zip(labels, probabilities)}
            
            # Determinar sentimiento predominante
            max_label = max(scores_dict, key=scores_dict.get)
            max_confidence = scores_dict[max_label]
            
            # Análisis contextual específico de montañismo
            mountaineering_context = self._analyze_mountaineering_context(text)
            
            result = {
                "route_id": route_id,
                "sentiment": max_label,
                "confidence": max_confidence,
                "scores": scores_dict,
                "text_length": len(text),
                "mountaineering_context": mountaineering_context,
                "analysis_timestamp": datetime.now().isoformat()
            }
            
            return result
            
        except Exception as e:
            print(f"❌ Error en análisis de sentimientos para ruta {route_id}: {e}")
            return self._empty_sentiment_result()
    
    def _analyze_mountaineering_context(self, text: str) -> Dict[str, Any]:
        """
        Análisis contextual específico de montañismo
        """
        text_lower = text.lower()
        
        context = {
            'risk_mentions': 0,
            'positive_experience': 0,
            'technical_difficulty': 0,
            'emotional_intensity': 0,
            'recommendation_tone': 0
        }
        
        # Contar menciones de riesgo
        risk_words = self.nlp_engine.mountaineering_vocab['risk_factors']
        context['risk_mentions'] = sum(1 for word in risk_words if word in text_lower)
        
        # Detectar experiencia positiva
        positive_words = ['espectacular', 'hermoso', 'recomendado', 'excelente', 'increíble']
        context['positive_experience'] = sum(1 for word in positive_words if word in text_lower)
        
        # Detectar dificultad técnica
        tech_words = ['técnico', 'difícil', 'complejo', 'desafiante', 'grado']
        context['technical_difficulty'] = sum(1 for word in tech_words if word in text_lower)
        
        # Intensidad emocional
        emotional_words = self.nlp_engine.mountaineering_vocab['emotional_indicators']
        context['emotional_intensity'] = sum(1 for word in emotional_words if word in text_lower)
        
        return context
    
    def _empty_sentiment_result(self) -> Dict[str, Any]:
        """Resultado vacío para casos de error"""
        return {
            "route_id": None,
            "sentiment": "neutral",
            "confidence": 0.0,
            "scores": {"neutral": 1.0},
            "text_length": 0,
            "mountaineering_context": {},
            "analysis_timestamp": datetime.now().isoformat()
        }
    
    def batch_analyze_sentiments(self, routes_df: pd.DataFrame, max_routes: int = 100) -> List[Dict]:
        """
        Análisis en lote de sentimientos
        """
        print(f"📊 Iniciando análisis de sentimientos para {min(len(routes_df), max_routes)} rutas...")
        
        results = []
        routes_to_process = routes_df.head(max_routes)
        
        for idx, route in routes_to_process.iterrows():
            if idx % 20 == 0:
                print(f"🔄 Procesando ruta {idx + 1}/{len(routes_to_process)}: {route['name'][:50]}...")
            
            result = self.analyze_route_sentiment(
                route['description_clean'], 
                route['route_id']
            )
            
            # Agregar información adicional de la ruta
            result.update({
                'route_name': route['name'],
                'elevation': route.get('elevation'),
                'location': route.get('location'),
                'first_ascent_year': route.get('first_ascent_year')
            })
            
            results.append(result)
            
            # Pausa pequeña para no sobrecargar
            time.sleep(0.1)
        
        print(f"✅ Análisis de sentimientos completado: {len(results)} rutas procesadas")
        return results

# Ejecutar análisis de sentimientos
sentiment_analyzer = SentimentAnalyzer(nlp_engine)
sentiment_results = sentiment_analyzer.batch_analyze_sentiments(routes_df, max_routes=150)

# Convertir a DataFrame para análisis
sentiment_df = pd.DataFrame(sentiment_results)
print(f"📊 Resultados de sentimientos: {len(sentiment_df)} análisis completados")

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.data

## 8. Extracción de Entidades Nombradas y Construcción de Redes Sociales

In [0]:
class MountaineerNetworkExtractor:
    """
    Extractor de redes sociales basado en análisis de entidades nombradas
    """
    
    def __init__(self, nlp_engine):
        self.nlp_engine = nlp_engine
        self.social_network = nx.Graph()
        self.mountaineer_mentions = {}
        self.location_networks = {}
        
    def extract_named_entities(self, text: str, route_id: str = None) -> Dict[str, List[Dict]]:
        """
        Extracción avanzada de entidades nombradas
        """
        if not text or len(text.strip()) < 10:
            return {"persons": [], "locations": [], "organizations": []}
        
        try:
            # Usar pipeline NER
            ner_pipeline = self.nlp_engine.models['ner_pipeline']
            ner_results = ner_pipeline(text[:512])  # Limitar longitud
            
            entities = {
                "persons": [],
                "locations": [],
                "organizations": []
            }
            
            # Procesar resultados NER
            for entity in ner_results:
                entity_type = entity['entity_group']
                entity_text = entity['word'].strip()
                confidence = entity['score']
                
                if confidence > 0.7:  # Filtro de confianza
                    if entity_type == 'PER' and len(entity_text) > 2:
                        entities["persons"].append({
                            "name": entity_text,
                            "confidence": confidence,
                            "route_id": route_id
                        })
                    elif entity_type in ['LOC', 'GPE'] and len(entity_text) > 2:
                        entities["locations"].append({
                            "name": entity_text,
                            "confidence": confidence,
                            "route_id": route_id
                        })
                    elif entity_type == 'ORG' and len(entity_text) > 2:
                        entities["organizations"].append({
                            "name": entity_text,
                            "confidence": confidence,
                            "route_id": route_id
                        })
            
            # Extracción adicional con patrones específicos de montañismo
            mountaineering_entities = self._extract_mountaineering_patterns(text, route_id)
            
            # Combinar resultados
            for category in entities:
                if category in mountaineering_entities:
                    entities[category].extend(mountaineering_entities[category])
            
            return entities
            
        except Exception as e:
            print(f"❌ Error en extracción de entidades para ruta {route_id}: {e}")
            return {"persons": [], "locations": [], "organizations": []}
    
    def _extract_mountaineering_patterns(self, text: str, route_id: str = None) -> Dict[str, List[Dict]]:
        """
        Extracción con patrones específicos de montañismo
        """
        entities = {"persons": [], "locations": [], "organizations": []}
        
        # Patrones para montañistas
        mountaineer_patterns = [
            r'(?:ascendido|realizado|escalado|logrado|completado)\s+por\s+([A-Z][a-záéíóúñ]+(?:\s+[A-Z][a-záéíóúñ]+)*)',
            r'(?:primer|primera)\s+ascen[sc]i[óo]n:?\s*([A-Z][a-záéíóúñ]+(?:\s+[A-Z][a-záéíóúñ]+)*)',
            r'([A-Z][a-záéíóúñ]+(?:\s+[A-Z][a-záéíóúñ]+)*)\s+(?:y|junto\s+a|con)\s+([A-Z][a-záéíóúñ]+(?:\s+[A-Z][a-záéíóúñ]+)*)'
        ]
        
        for pattern in mountaineer_patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                for group_idx in range(1, match.lastindex + 1 if match.lastindex else 1):
                    name = match.group(group_idx).strip()
                    if self._is_valid_person_name(name):
                        entities["persons"].append({
                            "name": name,
                            "confidence": 0.8,
                            "route_id": route_id,
                            "extraction_method": "pattern_matching"
                        })
        
        # Patrones para ubicaciones específicas
        location_patterns = [
            r'(?:volcán|cerro|monte|montaña|nevado|cordón|macizo)\s+([A-Z][a-záéíóúñ]+(?:\s+[A-Z][a-záéíóúñ]+)*)',
            r'(?:región|provincia|comuna)\s+(?:de\s+)?([A-Z][a-záéíóúñ]+(?:\s+[A-Z][a-záéíóúñ]+)*)',
            r'([A-Z][a-záéíóúñ]+(?:\s+[A-Z][a-záéíóúñ]+)*)\s+(?:Andes|Cordillera)'
        ]
        
        for pattern in location_patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                location = match.group(1).strip()
                if len(location) > 2:
                    entities["locations"].append({
                        "name": location,
                        "confidence": 0.7,
                        "route_id": route_id,
                        "extraction_method": "pattern_matching"
                    })
        
        return entities
    
    def _is_valid_person_name(self, name: str) -> bool:
        """
        Valida si un texto parece ser un nombre de persona
        """
        if not name or len(name) < 3 or len(name) > 50:
            return False
        
        # Filtrar palabras comunes que no son nombres
        invalid_words = {
            'el', 'la', 'los', 'las', 'de', 'del', 'y', 'con', 'por', 'para',
            'cerro', 'volcán', 'monte', 'montaña', 'región', 'provincia', 'comuna',
            'primer', 'primera', 'segundo', 'segunda', 'tercero', 'tercera',
            'año', 'años', 'metro', 'metros', 'kilómetro', 'kilómetros'
        }
        
        name_words = name.lower().split()
        if any(word in invalid_words for word in name_words):
            return False
        
        # Debe contener al menos una palabra con mayúscula inicial
        if not any(word[0].isupper() for word in name.split() if word):
            return False
        
        return True
    
    def build_social_network(self, entities_list: List[Dict]) -> nx.Graph:
        """
        Construye red social a partir de entidades extraídas
        """
        print("🔗 Construyendo red social de montañistas...")
        
        # Diccionario para agrupar montañistas por ruta
        route_mountaineers = {}
        
        for entity_data in entities_list:
            route_id = entity_data.get('route_id')
            persons = entity_data.get('persons', [])
            
            if route_id and persons:
                if route_id not in route_mountaineers:
                    route_mountaineers[route_id] = []
                
                for person in persons:
                    name = person['name']
                    confidence = person['confidence']
                    
                    # Agregar a lista de montañistas de la ruta
                    route_mountaineers[route_id].append({
                        'name': name,
                        'confidence': confidence
                    })
                    
                    # Agregar nodo al grafo si no existe
                    if not self.social_network.has_node(name):
                        self.social_network.add_node(name, 
                                                   type='mountaineer',
                                                   routes=[],
                                                   total_mentions=0,
                                                   avg_confidence=0.0)
                    
                    # Actualizar información del nodo
                    node_data = self.social_network.nodes[name]
                    node_data['routes'].append(route_id)
                    node_data['total_mentions'] += 1
                    
                    # Actualizar promedio de confianza
                    current_conf = node_data.get('avg_confidence', 0.0)
                    total_mentions = node_data['total_mentions']
                    node_data['avg_confidence'] = (current_conf * (total_mentions - 1) + confidence) / total_mentions
        
        # Crear conexiones entre montañistas que aparecen en la misma ruta
        for route_id, mountaineers in route_mountaineers.items():
            if len(mountaineers) > 1:
                for i, m1 in enumerate(mountaineers):
                    for m2 in mountaineers[i+1:]:
                        name1, name2 = m1['name'], m2['name']
                        
                        if self.social_network.has_edge(name1, name2):
                            # Incrementar peso de conexión existente
                            self.social_network[name1][name2]['weight'] += 1
                            self.social_network[name1][name2]['routes'].append(route_id)
                        else:
                            # Crear nueva conexión
                            self.social_network.add_edge(name1, name2,
                                                       weight=1,
                                                       routes=[route_id],
                                                       connection_type='co_mentioned')
        
        print(f"✅ Red social construida:")
        print(f"   - Montañistas: {self.social_network.number_of_nodes()}")
        print(f"   - Conexiones: {self.social_network.number_of_edges()}")
        
        return self.social_network
    
    def batch_extract_entities(self, routes_df: pd.DataFrame, max_routes: int = 100) -> List[Dict]:
        """
        Extracción en lote de entidades nombradas
        """
        print(f"🏷️ Iniciando extracción de entidades para {min(len(routes_df), max_routes)} rutas...")
        
        entities_list = []
        routes_to_process = routes_df.head(max_routes)
        
        for idx, route in routes_to_process.iterrows():
            if idx % 25 == 0:
                print(f"🔄 Procesando entidades {idx + 1}/{len(routes_to_process)}: {route['name'][:40]}...")
            
            entities = self.extract_named_entities(
                route['description_clean'], 
                route['route_id']
            )
            
            entities['route_id'] = route['route_id']
            entities['route_name'] = route['name']
            entities_list.append(entities)
            
            time.sleep(0.05)  # Pausa mínima
        
        print(f"✅ Extracción de entidades completada: {len(entities_list)} rutas procesadas")
        return entities_list

# Ejecutar extracción de entidades y construcción de red
network_extractor = MountaineerNetworkExtractor(nlp_engine)
entities_results = network_extractor.batch_extract_entities(routes_df, max_routes=150)

# Construir red social
social_network = network_extractor.build_social_network(entities_results)

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.data

## 9. Análisis Avanzado con LLM (Llama 4)

In [0]:
class AdvancedLLMAnalyzer:
    """
    Analizador avanzado usando LLM para insights profundos
    """
    
    def __init__(self, nlp_engine):
        self.nlp_engine = nlp_engine
        self.llm_client = nlp_engine.models.get('llm_client')
        
    def create_mountaineering_analysis_prompt(self, route_description: str, route_name: str = "") -> str:
        """
        Crea prompt especializado para análisis de montañismo
        """
        prompt = f"""
Analiza esta descripción de ruta de montañismo desde la perspectiva de REDES SOCIALES y devuelve un JSON estructurado:

RUTA: {route_name}
DESCRIPCIÓN: {route_description}

Enfócate en identificar:
1. COLABORACIONES SOCIALES entre montañistas
2. INFLUENCIA Y LIDERAZGO en la comunidad
3. TRANSMISIÓN DE CONOCIMIENTO entre generaciones
4. REDES DE CONFIANZA y reputación
5. COMUNIDADES ESPECIALIZADAS

RESPONDE SOLO CON ESTE JSON:
{{
    "redes_sociales": {{
        "montañistas_mencionados": ["nombre1", "nombre2"],
        "colaboraciones_detectadas": [
            {{"participantes": ["A", "B"], "tipo": "primer_ascenso|repeticion|guía"}},
        ],
        "indicadores_liderazgo": ["indicador1", "indicador2"],
        "transmision_conocimiento": {{
            "tipo": "enseñanza|mentoria|documentacion",
            "evidencia": "texto que lo demuestra"
        }}
    }},
    "analisis_comunidad": {{
        "tipo_comunidad": "elite|recreacional|técnica|familiar",
        "especialización": "alta_montaña|escalada|trekking|esquí",
        "nivel_colaboracion": 1-5,
        "indicadores_confianza": ["indicador1"]
    }},
    "contenido_social": {{
        "tono_comunicacion": "formal|informal|técnico|narrativo",
        "nivel_detalle": 1-5,
        "orientacion": "seguridad|aventura|técnica|turística",
        "audiencia_objetivo": "principiantes|intermedios|expertos"
    }},
    "patrones_influencia": {{
        "referencias_autoridad": ["referencia1"],
        "validacion_social": ["elemento1"],
        "reputacion_indicators": ["indicator1"]
    }},
    "metadatos": {{
        "confianza_analisis": 0.8,
        "complejidad_social": 1-5,
        "riqueza_informacion": 1-5
    }}
}}
"""
        return prompt
    
    def analyze_route_with_llm(self, route_description: str, route_name: str = "", route_id: str = None) -> Dict[str, Any]:
        """
        Análisis avanzado de ruta usando LLM
        """
        if not self.llm_client:
            return self._empty_llm_result()
        
        if not route_description or len(route_description.strip()) < 20:
            return self._empty_llm_result()
        
        try:
            prompt = self.create_mountaineering_analysis_prompt(route_description, route_name)
            
            response = self.llm_client.chat.completions.create(
                model="databricks-llama-4-maverick",
                messages=[
                    {"role": "system", "content": "Eres un experto en análisis de redes sociales aplicado al montañismo. Responde SOLO con JSON válido."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=2000
            )
            
            response_text = response.choices[0].message.content.strip()
            
            # Limpiar respuesta
            if response_text.startswith("```json"):
                response_text = response_text.replace("```json", "").replace("```", "").strip()
            
            # Parsear JSON
            try:
                result = json.loads(response_text)
                result['route_id'] = route_id
                result['route_name'] = route_name
                result['analysis_timestamp'] = datetime.now().isoformat()
                return result
            except json.JSONDecodeError as je:
                print(f"⚠️ Error JSON en ruta {route_id}: {je}")
                return self._create_fallback_analysis(route_description, route_name, route_id)
                
        except Exception as e:
            print(f"❌ Error LLM en ruta {route_id}: {e}")
            return self._empty_llm_result()
    
    def _create_fallback_analysis(self, description: str, route_name: str, route_id: str) -> Dict[str, Any]:
        """
        Análisis de respaldo cuando falla el LLM
        """
        # Análisis básico con regex y heurísticas
        desc_lower = description.lower()
        
        # Detectar colaboraciones básicas
        collaboration_patterns = [
            r'con\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)',
            r'junto\s+a\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)',
            r'acompañado\s+de\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)'
        ]
        
        mountaineers = []
        for pattern in collaboration_patterns:
            matches = re.findall(pattern, description)
            mountaineers.extend(matches)
        
        # Determinar tipo de comunidad
        if any(word in desc_lower for word in ['técnico', 'grado', 'escalada']):
            community_type = 'técnica'
        elif any(word in desc_lower for word in ['familia', 'niños', 'principiante']):
            community_type = 'familiar'
        elif any(word in desc_lower for word in ['extremo', 'difícil', 'experto']):
            community_type = 'elite'
        else:
            community_type = 'recreacional'
        
        return {
            "redes_sociales": {
                "montañistas_mencionados": mountaineers[:5],
                "colaboraciones_detectadas": [],
                "indicadores_liderazgo": [],
                "transmision_conocimiento": {"tipo": "documentacion", "evidencia": ""}
            },
            "analisis_comunidad": {
                "tipo_comunidad": community_type,
                "especialización": "general",
                "nivel_colaboracion": 2,
                "indicadores_confianza": []
            },
            "contenido_social": {
                "tono_comunicacion": "informal",
                "nivel_detalle": 3,
                "orientacion": "general",
                "audiencia_objetivo": "intermedios"
            },
            "patrones_influencia": {
                "referencias_autoridad": [],
                "validacion_social": [],
                "reputacion_indicators": []
            },
            "metadatos": {
                "confianza_analisis": 0.4,
                "complejidad_social": 2,
                "riqueza_informacion": 2
            },
            "route_id": route_id,
            "route_name": route_name,
            "analysis_timestamp": datetime.now().isoformat(),
            "fallback_analysis": True
        }
    
    def _empty_llm_result(self) -> Dict[str, Any]:
        """Resultado vacío para casos de error"""
        return {
            "redes_sociales": {"montañistas_mencionados": [], "colaboraciones_detectadas": [], "indicadores_liderazgo": [], "transmision_conocimiento": {"tipo": "", "evidencia": ""}},
            "analisis_comunidad": {"tipo_comunidad": "desconocido", "especialización": "general", "nivel_colaboracion": 0, "indicadores_confianza": []},
            "contenido_social": {"tono_comunicacion": "desconocido", "nivel_detalle": 0, "orientacion": "general", "audiencia_objetivo": "desconocido"},
            "patrones_influencia": {"referencias_autoridad": [], "validacion_social": [], "reputacion_indicators": []},
            "metadatos": {"confianza_analisis": 0.0, "complejidad_social": 0, "riqueza_informacion": 0},
            "analysis_timestamp": datetime.now().isoformat()
        }
    
    def batch_analyze_with_llm(self, routes_df: pd.DataFrame, max_routes: int = 50) -> List[Dict]:
        """
        Análisis en lote con LLM
        """
        print(f"🚀 Iniciando análisis LLM para {min(len(routes_df), max_routes)} rutas...")
        
        results = []
        routes_to_process = routes_df.head(max_routes)
        
        for idx, route in routes_to_process.iterrows():
            print(f"🧠 Analizando con LLM {idx + 1}/{len(routes_to_process)}: {route['name'][:40]}...")
            
            result = self.analyze_route_with_llm(
                route['description_clean'], 
                route['name'],
                route['route_id']
            )
            
            results.append(result)
            
            # Pausa entre llamadas LLM
            time.sleep(2)
        
        print(f"✅ Análisis LLM completado: {len(results)} rutas procesadas")
        return results

# Ejecutar análisis LLM
llm_analyzer = AdvancedLLMAnalyzer(nlp_engine)
llm_results = llm_analyzer.batch_analyze_with_llm(routes_df, max_routes=4000)

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.data

## 10. Análisis de Métricas de Red Social

In [0]:
class SocialNetworkAnalyzer:
    """
    Analizador especializado de métricas de redes sociales
    """
    
    def __init__(self, network: nx.Graph):
        self.network = network
        self.metrics = {}
        
    def calculate_comprehensive_metrics(self) -> Dict[str, Any]:
        """
        Calcula métricas completas de la red social
        """
        print("📊 Calculando métricas de red social...")
        
        if self.network.number_of_nodes() == 0:
            print("⚠️ Red vacía, no se pueden calcular métricas")
            return {}
        
        metrics = {}
        
        # === MÉTRICAS BÁSICAS ===
        metrics['basic'] = {
            'num_nodes': self.network.number_of_nodes(),
            'num_edges': self.network.number_of_edges(),
            'density': nx.density(self.network),
            'is_connected': nx.is_connected(self.network)
        }
        
        print(f"   Nodos: {metrics['basic']['num_nodes']}, Aristas: {metrics['basic']['num_edges']}")
        
        # === MÉTRICAS DE CENTRALIDAD ===
        if metrics['basic']['num_nodes'] > 1:
            print("   📈 Calculando centralidades...")
            
            centrality_degree = nx.degree_centrality(self.network)
            centrality_betweenness = nx.betweenness_centrality(self.network)
            centrality_closeness = nx.closeness_centrality(self.network)
            centrality_eigenvector = self._safe_eigenvector_centrality()
            
            metrics['centrality'] = {
                'degree': centrality_degree,
                'betweenness': centrality_betweenness,
                'closeness': centrality_closeness,
                'eigenvector': centrality_eigenvector,
                'top_degree': self._get_top_nodes(centrality_degree, 10),
                'top_betweenness': self._get_top_nodes(centrality_betweenness, 10),
                'top_closeness': self._get_top_nodes(centrality_closeness, 10)
            }
        
        # === ANÁLISIS DE COMUNIDADES ===
        if metrics['basic']['num_nodes'] > 2:
            print("   👥 Detectando comunidades...")
            communities = self._detect_communities()
            metrics['communities'] = communities
        
        # === MÉTRICAS DE CONECTIVIDAD ===
        components = list(nx.connected_components(self.network))
        metrics['connectivity'] = {
            'num_components': len(components),
            'largest_component_size': len(max(components, key=len)) if components else 0,
            'component_sizes': [len(comp) for comp in components]
        }
        
        # === ANÁLISIS DE DISTRIBUCIÓN DE GRADOS ===
        degrees = [d for n, d in self.network.degree()]
        if degrees:
            metrics['degree_distribution'] = {
                'mean': np.mean(degrees),
                'std': np.std(degrees),
                'max': max(degrees),
                'min': min(degrees),
                'median': np.median(degrees)
            }
        
        # === CLUSTERING ===
        try:
            avg_clustering = nx.average_clustering(self.network)
            metrics['clustering'] = {
                'average_clustering': avg_clustering,
                'global_clustering': nx.transitivity(self.network)
            }
        except:
            metrics['clustering'] = {'average_clustering': 0, 'global_clustering': 0}
        
        print("✅ Métricas de red calculadas exitosamente")
        self.metrics = metrics
        return metrics
    
    def _safe_eigenvector_centrality(self) -> Dict:
        """
        Cálculo seguro de centralidad de eigenvector
        """
        try:
            return nx.eigenvector_centrality(self.network, max_iter=1000, tol=1e-06)
        except:
            print("   ⚠️ No se pudo calcular centralidad de eigenvector")
            return {}
    
    def _get_top_nodes(self, centrality_dict: Dict, top_k: int = 10) -> List[Tuple]:
        """
        Obtiene los top K nodos por centralidad
        """
        return sorted(centrality_dict.items(), key=lambda x: x[1], reverse=True)[:top_k]
    
    def _detect_communities(self) -> Dict[str, Any]:
        """
        Detección de comunidades usando múltiples algoritmos
        """
        communities_info = {}
        
        try:
            # Método 1: Louvain
            import networkx.algorithms.community as nx_comm
            louvain_communities = nx_comm.louvain_communities(self.network)
            
            communities_info['louvain'] = {
                'num_communities': len(louvain_communities),
                'communities': [list(comm) for comm in louvain_communities],
                'modularity': nx_comm.modularity(self.network, louvain_communities),
                'sizes': [len(comm) for comm in louvain_communities]
            }
            
        except Exception as e:
            print(f"   ⚠️ Error en detección de comunidades: {e}")
            communities_info['louvain'] = {'num_communities': 0, 'communities': []}
        
        return communities_info
    
    def identify_key_players(self) -> Dict[str, List[Dict]]:
        """
        Identifica actores clave en la red
        """
        if not self.metrics:
            self.calculate_comprehensive_metrics()
        
        key_players = {
            'influencers': [],  # Alta centralidad de grado
            'connectors': [],   # Alta centralidad de intermediación
            'central_figures': [],  # Alta centralidad de cercanía
            'hub_nodes': []     # Nodos con muchas conexiones
        }
        
        if 'centrality' in self.metrics:
            centrality = self.metrics['centrality']
            
            # Influencers (alto grado)
            for name, score in centrality['top_degree'][:5]:
                node_data = self.network.nodes[name]
                key_players['influencers'].append({
                    'name': name,
                    'degree_centrality': score,
                    'total_routes': len(node_data.get('routes', [])),
                    'total_mentions': node_data.get('total_mentions', 0)
                })
            
            # Connectors (alta intermediación)
            for name, score in centrality['top_betweenness'][:5]:
                if score > 0:
                    key_players['connectors'].append({
                        'name': name,
                        'betweenness_centrality': score,
                        'role': 'bridge_builder'
                    })
            
            # Figuras centrales (alta cercanía)
            for name, score in centrality['top_closeness'][:5]:
                key_players['central_figures'].append({
                    'name': name,
                    'closeness_centrality': score,
                    'accessibility': 'high'
                })
        
        return key_players
    
    def analyze_network_evolution(self, temporal_data: List[Dict] = None) -> Dict[str, Any]:
        """
        Análisis de evolución temporal de la red (si hay datos temporales)
        """
        evolution_analysis = {
            'growth_pattern': 'steady',  # Placeholder
            'peak_periods': [],
            'influential_eras': {},
            'community_changes': {}
        }
        
        # TODO: Implementar análisis temporal real cuando haya datos de fechas
        
        return evolution_analysis

# Ejecutar análisis de red social
if social_network.number_of_nodes() > 0:
    network_analyzer = SocialNetworkAnalyzer(social_network)
    network_metrics = network_analyzer.calculate_comprehensive_metrics()
    key_players = network_analyzer.identify_key_players()
    
    print(f"\n🎯 RESUMEN DE RED SOCIAL:")
    print(f"   - Montañistas identificados: {network_metrics['basic']['num_nodes']}")
    print(f"   - Colaboraciones detectadas: {network_metrics['basic']['num_edges']}")
    print(f"   - Densidad de red: {network_metrics['basic']['density']:.4f}")
    print(f"   - Comunidades detectadas: {network_metrics.get('communities', {}).get('louvain', {}).get('num_communities', 0)}")
else:
    print("⚠️ No se detectaron suficientes conexiones sociales para análisis de red")
    network_metrics = {}
    key_players = {}

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.data

## 11. Almacenamiento de Resultados en Delta Lake

In [0]:
def save_analysis_results_to_delta():
    """
    Guarda todos los resultados de análisis en tablas Delta Lake
    """
    print("💾 Guardando resultados de análisis en Delta Lake...")
    
    # === TABLA 1: ANÁLISIS DE SENTIMIENTOS ===
    if sentiment_results:
        print("📊 Guardando análisis de sentimientos...")
        
        # Preparar datos de sentimientos de forma aplanada
        sentiment_flat = []
        for result in sentiment_results:
            # Extraer scores individuales para evitar conflictos
            scores_dict = result.get('scores', {})
            
            flat_record = {
                'route_id': result.get('route_id'),
                'route_name': result.get('route_name'),
                'sentiment': result.get('sentiment'),
                'confidence': result.get('confidence'),
                'text_length': result.get('text_length'),
                'analysis_timestamp': result.get('analysis_timestamp'),
                'elevation': result.get('elevation'),
                'location': result.get('location'),
                'first_ascent_year': result.get('first_ascent_year'),
                
                # Scores individuales con nombres únicos
                'score_positivo': scores_dict.get('positivo', scores_dict.get('POSITIVE', 0.0)),
                'score_negativo': scores_dict.get('negativo', scores_dict.get('NEGATIVE', 0.0)),
                'score_neutro': scores_dict.get('neutro', scores_dict.get('NEUTRAL', scores_dict.get('neutral', 0.0))),
                
                # Contexto de montañismo
                'risk_mentions': result.get('mountaineering_context', {}).get('risk_mentions', 0),
                'positive_experience': result.get('mountaineering_context', {}).get('positive_experience', 0),
                'technical_difficulty': result.get('mountaineering_context', {}).get('technical_difficulty', 0),
                'emotional_intensity': result.get('mountaineering_context', {}).get('emotional_intensity', 0),
                'recommendation_tone': result.get('mountaineering_context', {}).get('recommendation_tone', 0)
            }
            
            sentiment_flat.append(flat_record)
        
        # Crear DataFrame y guardar
        sentiment_df_clean = pd.DataFrame(sentiment_flat)
        sentiment_spark_df = spark.createDataFrame(sentiment_df_clean)
        
        # Eliminar tabla existente si existe
        spark.sql("DROP TABLE IF EXISTS andes_sentiment_analysis")
        
        # Guardar nueva tabla
        sentiment_spark_df.write.format("delta").mode("overwrite").saveAsTable("andes_sentiment_analysis")
        print(f"✅ Tabla 'andes_sentiment_analysis' creada: {len(sentiment_flat)} registros")
    
    # === TABLA 2: ENTIDADES NOMBRADAS ===
    if entities_results:
        print("🏷️ Guardando entidades nombradas...")
        
        # Aplanar estructura de entidades
        entities_flat = []
        for entity_data in entities_results:
            route_id = entity_data['route_id']
            route_name = entity_data['route_name']
            
            for entity_type in ['persons', 'locations', 'organizations']:
                for entity in entity_data.get(entity_type, []):
                    entities_flat.append({
                        'route_id': route_id,
                        'route_name': route_name,
                        'entity_type': entity_type,
                        'entity_name': entity['name'],
                        'confidence': entity['confidence'],
                        'extraction_method': entity.get('extraction_method', 'ner_model')
                    })
        
        if entities_flat:
            entities_df = pd.DataFrame(entities_flat)
            entities_spark_df = spark.createDataFrame(entities_df)
            spark.sql("DROP TABLE IF EXISTS andes_named_entities")
            entities_spark_df.write.format("delta").mode("overwrite").saveAsTable("andes_named_entities")
            print(f"✅ Tabla 'andes_named_entities' creada: {len(entities_flat)} entidades")
    
    # === TABLA 3: ANÁLISIS LLM ===
    if llm_results:
        print("🚀 Guardando análisis LLM...")
        
        # Función helper para limpiar valores
        def clean_value(value, default_value, value_type=str):
            """Limpia valores para evitar problemas de conversión"""
            if value is None or (isinstance(value, str) and value.strip() == ''):
                return default_value
            if value_type == str:
                return str(value).strip()
            elif value_type == int:
                try:
                    return int(float(value)) if value != '' else default_value
                except (ValueError, TypeError):
                    return default_value
            elif value_type == float:
                try:
                    return float(value) if value != '' else default_value
                except (ValueError, TypeError):
                    return default_value
            return value
        
        # Aplanar estructura compleja de LLM
        llm_flat = []
        for llm_data in llm_results:
            # Información básica con limpieza
            base_info = {
                'route_id': clean_value(llm_data.get('route_id'), 'unknown'),
                'route_name': clean_value(llm_data.get('route_name'), 'unknown'),
                'analysis_timestamp': clean_value(llm_data.get('analysis_timestamp'), datetime.now().isoformat()),
                'fallback_analysis': bool(llm_data.get('fallback_analysis', False))
            }
            
            # Métricas del análisis de comunidad con limpieza
            community_analysis = llm_data.get('analisis_comunidad', {})
            content_analysis = llm_data.get('contenido_social', {})
            metadata = llm_data.get('metadatos', {})
            
            base_info.update({
                'tipo_comunidad': clean_value(community_analysis.get('tipo_comunidad'), 'desconocido'),
                'especializacion': clean_value(community_analysis.get('especialización'), 'general'),
                'nivel_colaboracion': clean_value(community_analysis.get('nivel_colaboracion'), 0, int),
                'tono_comunicacion': clean_value(content_analysis.get('tono_comunicacion'), 'desconocido'),
                'audiencia_objetivo': clean_value(content_analysis.get('audiencia_objetivo'), 'desconocido'),
                'confianza_analisis': clean_value(metadata.get('confianza_analisis'), 0.0, float),
                'complejidad_social': clean_value(metadata.get('complejidad_social'), 0, int),
                'riqueza_informacion': clean_value(metadata.get('riqueza_informacion'), 0, int)
            })
            
            # Montañistas mencionados (como JSON string)
            mountaineers = llm_data.get('redes_sociales', {}).get('montañistas_mencionados', [])
            base_info['montañistas_mencionados'] = json.dumps(mountaineers if mountaineers else [], ensure_ascii=False)
            
            # Colaboraciones (como JSON string)
            collaborations = llm_data.get('redes_sociales', {}).get('colaboraciones_detectadas', [])
            base_info['colaboraciones_detectadas'] = json.dumps(collaborations if collaborations else [], ensure_ascii=False)
            
            llm_flat.append(base_info)
        
        if llm_flat:
            llm_df = pd.DataFrame(llm_flat)
            
            # Asegurar tipos de datos correctos
            llm_df = llm_df.fillna({
                'route_id': 'unknown',
                'route_name': 'unknown', 
                'tipo_comunidad': 'desconocido',
                'especializacion': 'general',
                'tono_comunicacion': 'desconocido',
                'audiencia_objetivo': 'desconocido',
                'montañistas_mencionados': '[]',
                'colaboraciones_detectadas': '[]'
            })
            
            # Convertir a tipos apropiados
            llm_df['nivel_colaboracion'] = pd.to_numeric(llm_df['nivel_colaboracion'], errors='coerce').fillna(0).astype(int)
            llm_df['confianza_analisis'] = pd.to_numeric(llm_df['confianza_analisis'], errors='coerce').fillna(0.0)
            llm_df['complejidad_social'] = pd.to_numeric(llm_df['complejidad_social'], errors='coerce').fillna(0).astype(int)
            llm_df['riqueza_informacion'] = pd.to_numeric(llm_df['riqueza_informacion'], errors='coerce').fillna(0).astype(int)
            
            llm_spark_df = spark.createDataFrame(llm_df)
            spark.sql("DROP TABLE IF EXISTS andes_llm_analysis")
            llm_spark_df.write.format("delta").mode("overwrite").saveAsTable("andes_llm_analysis")
            print(f"✅ Tabla 'andes_llm_analysis' creada: {len(llm_flat)} análisis")
    
    # === TABLA 4: MÉTRICAS DE RED SOCIAL ===
    if network_metrics:
        print("🔗 Guardando métricas de red social...")
        
        # Crear tabla con métricas agregadas
        network_summary = [{
            'analysis_timestamp': datetime.now().isoformat(),
            'num_nodes': network_metrics['basic']['num_nodes'],
            'num_edges': network_metrics['basic']['num_edges'],
            'density': network_metrics['basic']['density'],
            'is_connected': network_metrics['basic']['is_connected'],
            'num_components': network_metrics['connectivity']['num_components'],
            'largest_component_size': network_metrics['connectivity']['largest_component_size'],
            'average_clustering': network_metrics.get('clustering', {}).get('average_clustering', 0),
            'num_communities': network_metrics.get('communities', {}).get('louvain', {}).get('num_communities', 0),
            'modularity': network_metrics.get('communities', {}).get('louvain', {}).get('modularity', 0)
        }]
        
        network_summary_df = pd.DataFrame(network_summary)
        network_summary_spark_df = spark.createDataFrame(network_summary_df)
        spark.sql("DROP TABLE IF EXISTS andes_network_metrics")
        network_summary_spark_df.write.format("delta").mode("overwrite").saveAsTable("andes_network_metrics")
        print(f"✅ Tabla 'andes_network_metrics' creada")
        
        # Guardar centralidades individuales
        if 'centrality' in network_metrics and social_network.number_of_nodes() > 0:
            centrality_records = []
            for node, degree_cent in network_metrics['centrality']['degree'].items():
                record = {
                    'mountaineer_name': node,
                    'degree_centrality': degree_cent,
                    'betweenness_centrality': network_metrics['centrality']['betweenness'].get(node, 0),
                    'closeness_centrality': network_metrics['centrality']['closeness'].get(node, 0),
                    'total_routes': len(social_network.nodes[node].get('routes', [])),
                    'total_mentions': social_network.nodes[node].get('total_mentions', 0),
                    'avg_confidence': social_network.nodes[node].get('avg_confidence', 0.0)
                }
                centrality_records.append(record)
            
            if centrality_records:
                centrality_df = pd.DataFrame(centrality_records)
                centrality_spark_df = spark.createDataFrame(centrality_df)
                spark.sql("DROP TABLE IF EXISTS andes_mountaineer_centrality")
                centrality_spark_df.write.format("delta").mode("overwrite").saveAsTable("andes_mountaineer_centrality")
                print(f"✅ Tabla 'andes_mountaineer_centrality' creada: {len(centrality_records)} montañistas")
    
    print("💾 Todos los resultados guardados en Delta Lake exitosamente!")

# Ejecutar guardado corregido
save_analysis_results_to_delta()

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.data

## 12. Visualización y Dashboard de Resultados

In [0]:
def create_comprehensive_dashboard():
    """
    Crea dashboard completo de visualizaciones
    """
    print("📊 Generando Dashboard Integral de Análisis de Redes Sociales")
    print("="*70)
    
    # === GRÁFICO 1: DISTRIBUCIÓN DE SENTIMIENTOS ===
    if sentiment_results:
        sentiment_counts = sentiment_df['sentiment'].value_counts()
        
        fig1 = px.pie(
            values=sentiment_counts.values,
            names=sentiment_counts.index,
            title="Distribución de Sentimientos en Descripciones de Rutas",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig1.update_traces(textposition='inside', textinfo='percent+label')
        fig1.show()
        
        print(f"📈 Sentimientos analizados: {len(sentiment_df)} rutas")
        print(f"   - Positivo: {sentiment_counts.get('positivo', 0)}")
        print(f"   - Neutro: {sentiment_counts.get('neutro', 0)}")
        print(f"   - Negativo: {sentiment_counts.get('negativo', 0)}")
    
    # === GRÁFICO 2: RED SOCIAL DE MONTAÑISTAS ===
    if social_network.number_of_nodes() > 0:
        print(f"\n🔗 Visualizando red social ({social_network.number_of_nodes()} nodos)...")
        
        # Crear visualización de red con plotly
        pos = nx.spring_layout(social_network, k=2, iterations=50)
        
        # Nodos
        node_x = [pos[node][0] for node in social_network.nodes()]
        node_y = [pos[node][1] for node in social_network.nodes()]
        node_text = list(social_network.nodes())
        node_sizes = [social_network.degree(node) * 10 + 10 for node in social_network.nodes()]
        
        # Aristas
        edge_x = []
        edge_y = []
        for edge in social_network.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
        
        # Crear gráfico
        fig2 = go.Figure()
        
        # Agregar aristas
        fig2.add_trace(go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5, color='lightgray'),
            hoverinfo='none',
            mode='lines',
            name='Colaboraciones'
        ))
        
        # Agregar nodos
        fig2.add_trace(go.Scatter(
            x=node_x, y=node_y,
            mode='markers+text',
            marker=dict(
                size=node_sizes,
                color='lightblue',
                line=dict(width=2, color='darkblue')
            ),
            text=node_text,
            textposition="middle center",
            hovertemplate='<b>%{text}</b><br>Conexiones: %{marker.size}<extra></extra>',
            name='Montañistas'
        ))
        
        fig2.update_layout(
            title="Red Social de Montañistas - Colaboraciones Detectadas",
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20,l=5,r=5,t=40),
            annotations=[ dict(
                text="Tamaño del nodo = número de colaboraciones",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.005, y=-0.002,
                xanchor='left', yanchor='bottom',
                font=dict(color='gray', size=12)
            )],
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            height=600
        )
        
        fig2.show()
    
    # === GRÁFICO 3: TOP MONTAÑISTAS INFLUYENTES ===
    if key_players:
        print(f"\n🏔️ Top Montañistas Más Influyentes:")
        
        influencers = key_players.get('influencers', [])[:10]
        if influencers:
            names = [inf['name'] for inf in influencers]
            scores = [inf['degree_centrality'] for inf in influencers]
            
            fig3 = px.bar(
                x=scores, y=names,
                orientation='h',
                title="Top 10 Montañistas por Centralidad de Grado",
                labels={'x': 'Centralidad de Grado', 'y': 'Montañista'},
                color=scores,
                color_continuous_scale='Viridis'
            )
            fig3.update_layout(height=500, yaxis={'categoryorder':'total ascending'})
            fig3.show()
            
            # Imprimir ranking
            for i, inf in enumerate(influencers, 1):
                print(f"   {i:2d}. {inf['name']:<25} | Centralidad: {inf['degree_centrality']:.3f} | Rutas: {inf['total_routes']}")
    
    # === GRÁFICO 4: ANÁLISIS LLM - TIPOS DE COMUNIDAD ===
    if llm_results:
        community_types = [result.get('analisis_comunidad', {}).get('tipo_comunidad', 'desconocido') 
                          for result in llm_results]
        community_counts = pd.Series(community_types).value_counts()
        
        fig4 = px.bar(
            x=community_counts.index,
            y=community_counts.values,
            title="Tipos de Comunidad Detectados por LLM",
            labels={'x': 'Tipo de Comunidad', 'y': 'Número de Rutas'},
            color=community_counts.values,
            color_continuous_scale='Blues'
        )
        fig4.show()
        
        print(f"\n🧠 Análisis LLM - Tipos de Comunidad:")
        for comm_type, count in community_counts.items():
            print(f"   - {comm_type.capitalize()}: {count} rutas")
    
    # === TABLA RESUMEN ===
    print(f"\n📋 RESUMEN INTEGRAL DEL ANÁLISIS")
    print("="*50)
    print(f"📊 Rutas procesadas: {len(routes_df)}")
    print(f"🧠 Análisis de sentimientos: {len(sentiment_results) if sentiment_results else 0}")
    print(f"🏷️ Entidades extraídas: {sum(len(e.get('persons', [])) + len(e.get('locations', [])) for e in entities_results) if entities_results else 0}")
    print(f"🚀 Análisis LLM: {len(llm_results) if llm_results else 0}")
    print(f"🔗 Red social: {social_network.number_of_nodes()} nodos, {social_network.number_of_edges()} conexiones")
    
    if network_metrics:
        print(f"📈 Densidad de red: {network_metrics['basic']['density']:.4f}")
        print(f"👥 Comunidades: {network_metrics.get('communities', {}).get('louvain', {}).get('num_communities', 0)}")

# Ejecutar dashboard
create_comprehensive_dashboard()

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.data

## 14. Conclusiones

## 14. Conclusiones

---

Este proyecto demuestra que es posible aplicar técnicas avanzadas de análisis de redes sociales, procesamiento de lenguaje natural y modelos de lenguaje grandes para extraer insights valiosos sobre comunidades especializadas a partir de contenido textual no estructurado.

La metodología desarrollada es:
- **Escalable:** Procesa miles de documentos.
- **Reproducible:** Código abierto y documentado.
- **Ética:** Considera privacidad y responsabilidad social.
- **Práctica:** Resultados aplicables de inmediato.
- **Académica:** Contribuye al conocimiento científico.

Los resultados obtenidos sientan una base sólida para el desarrollo de aplicaciones que beneficien a la comunidad montañista chilena y sirvan de modelo para otras comunidades especializadas.

---

📋 **DETALLES TÉCNICOS Y ESTADÍSTICAS DEL PROYECTO**
============================================================

🔧 **ESPECIFICACIONES TÉCNICAS:**
   • Plataforma: Databricks Community Edition  
   • Lenguaje Principal: Python 3.9+  
   • Framework de Datos: Apache Spark 3.4+  
   • Almacenamiento: Delta Lake  
   • Modelos NLP: Transformers (Hugging Face)  
   • LLM: Databricks Llama 4 Maverick  
   • Librerías de Red: NetworkX 3.0+  
   • Visualización: Plotly, Matplotlib, Seaborn  

📊 **ESTADÍSTICAS DE PROCESAMIENTO:**
   • Rutas Procesadas: 3142  
   • Análisis de Sentimientos: 150  
   • Entidades Extraídas: 2031  
   • Análisis LLM: 50  
   • Red Social: 366 nodos, 877 conexiones  
   • Tiempo Total de Procesamiento: ~2-3 horas  
   • Volumen de Datos Generado: ~50 MB  

🔗 **FUENTES DE DATOS:**
   • Principal: AndesHandbook (www.andeshandbook.org)  
   • Tabla Base: andes_handbook_routes (datos previamente extraídos)  
   • Modelos NLP: Hugging Face Model Hub  
   • LLM: Databricks Model Serving  

🛠️ **TECNOLOGÍAS UTILIZADAS:**
   • NLP: transformers, torch, scipy  
   • Análisis de Redes: networkx  
   • Procesamiento: pandas, numpy, PySpark  
   • Visualización: plotly, matplotlib, seaborn  
   • Almacenamiento: Delta Lake, Spark SQL  

 

